# Run dataset based tests

Use the Validmind Developer Framework's `run_test` function to run built-in or custom tests that take any dataset or model as input. These tests generate outputs in the form of text, tables, and images that get populated in model documentation.

You'll learn how to:

- Find tests 
- Initialize a ValidMind dataset 
- Pass the dataset to the `run_test` fuction for any test that takes a `dataset` input

**We recommended that you first complete the [Explore tests](../explore_tests.ipynb) notebook,** to understand the basics of how to find and describe all the available tests in the developer framework before moving on to this advanced guide.

This interactive notebook provides a step-by-step guide for listing and filtering available tests, building a sample dataset, initializing the required ValidMind objects, running the test, and then logging the results to ValidMind. 

## Contents    
- [About ValidMind](#toc1_)    
  - [Before you begin](#toc1_1_)    
  - [New to ValidMind?](#toc1_2_)    
  - [Key concepts](#toc1_3_)    
- [Install the client library](#toc2_)    
- [Initialize the client library](#toc3_)    
- [List and filter available tests](#toc4_)    
- [Create a sample dataset](#toc5_)    
  - [Initialize a ValidMind dataset](#toc5_1_)    
  - [Run tests with the dataset](#toc5_2_)    
    - [Run a test that accepts parameters](#toc5_2_1_)    
- [Add test results to documentation](#toc6_)    
- [Next steps](#toc7_)    
  - [Discover more learning resources](#toc7_1_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=2
	maxLevel=4
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

<a id='toc1_'></a>

## About ValidMind
ValidMind is a platform for managing model risk, including risk associated with AI and statistical models.

You use the ValidMind Developer Framework to automate documentation and validation tests, and then use the ValidMind AI Risk Platform UI to collaborate on model documentation. Together, these products simplify model risk management, facilitate compliance with regulations and institutional standards, and enhance collaboration between yourself and model validators.

<a id='toc1_1_'></a>

### Before you begin
This notebook assumes you have basic familiarity with Python, including an understanding of how functions work. If you are new to Python, you can still run the notebook but we recommend further familiarizing yourself with the language.

If you encounter errors due to missing modules in your Python environment, install the modules with `pip install`, and then re-run the notebook. For more help, refer to [Installing Python Modules](https://docs.python.org/3/installing/index.html).

<a id='toc1_2_'></a>

### New to ValidMind?
If you haven't already seen our [Get started with the ValidMind Developer Framework](https://docs.validmind.ai/guide/get-started-developer-framework.html), we recommend you begin by exploring the available resources in this section. There, you can learn more about documenting models, find code samples, or read our developer reference.

::: {.callout-tip}

For access to all features available in this notebook, create a free ValidMind account.

Signing up is FREE — [**Sign up now!**](https://app.prod.validmind.ai)
:::

<a id='toc1_3_'></a>

### Key concepts

**Model documentation**: A structured and detailed record pertaining to a model, encompassing key components such as its underlying assumptions, methodologies, data sources, inputs, performance metrics, evaluations, limitations, and intended uses. It serves to ensure transparency, adherence to regulatory requirements, and a clear understanding of potential risks associated with the model’s application.

**Documentation template**: Functions as a test suite and lays out the structure of model documentation, segmented into various sections and sub-sections. Documentation templates define the structure of your model documentation, specifying the tests that should be run, and how the results should be displayed.

**Tests**: A function contained in the ValidMind Developer Framework, designed to run a specific quantitative test on the dataset or model. Tests are the building blocks of ValidMind, used to evaluate and document models and datasets, and can be run individually or as part of a suite defined by your model documentation template.

**Metrics**: A subset of tests that do not have thresholds. In the context of this notebook, metrics and tests can be thought of as interchangeable concepts.

**Custom metrics**: Custom metrics are functions that you define to evaluate your model or dataset. These functions can be registered with ValidMind to be used in the platform.

**Inputs**: Objects to be evaluated and documented in the ValidMind framework. They can be any of the following:

- **model**: A single model that has been initialized in ValidMind with [`vm.init_model()`](https://docs.validmind.ai/validmind/validmind.html#init_model).
- **dataset**: Single dataset that has been initialized in ValidMind with [`vm.init_dataset()`](https://docs.validmind.ai/validmind/validmind.html#init_dataset).
- **models**: A list of ValidMind models - usually this is used when you want to compare multiple models in your custom metric.
- **datasets**: A list of ValidMind datasets - usually this is used when you want to compare multiple datasets in your custom metric. See this [example](https://docs.validmind.ai/notebooks/how_to/run_tests_that_require_multiple_datasets.html) for more information.

**Parameters**: Additional arguments that can be passed when running a ValidMind test, used to pass additional information to a metric, customize its behavior, or provide additional context.

**Outputs**: Custom metrics can return elements like tables or plots. Tables may be a list of dictionaries (each representing a row) or a pandas DataFrame. Plots may be matplotlib or plotly figures.

**Test suites**: Collections of tests designed to run together to automate and generate model documentation end-to-end for specific use-cases.

Example: The [`classifier_full_suite`](https://docs.validmind.ai/validmind/validmind/test_suites/classifier.html#ClassifierFullSuite) test suite runs tests from the [`tabular_dataset`](https://docs.validmind.ai/validmind/validmind/test_suites/tabular_datasets.html) and [`classifier`](https://docs.validmind.ai/validmind/validmind/test_suites/classifier.html) test suites to fully document the data and model sections for binary classification model use-cases.

<a id='toc2_'></a>

## Install the client library

The client library provides Python support for the ValidMind Developer Framework. To install it:

In [ ]:
%pip install -q validmind

<a id='toc3_'></a>

## Initialize the client library

ValidMind generates a unique _code snippet_ for each registered model to connect with your developer environment. You initialize the client library with this code snippet, which ensures that your documentation and tests are uploaded to the correct model when you run the notebook.

Get your code snippet:

1. In a browser, log into the [Platform UI](https://app.prod.validmind.ai).

2. In the left sidebar, navigate to **Model Inventory** and click **+ Register new model**.

3. Enter the model details below and click **Continue**. ([Need more help?](https://docs.validmind.ai/guide/register-models-in-model-inventory.html))

    - Documentation template: `Binary Classification`
    - Use case: `Marketing - Attrition/Churn Management`

    You can fill in the other options according to your preference. 

4. Go to **Getting Started** and click **Copy snippet to clipboard**.

Next, replace this placeholder with your own code snippet:


In [ ]:
# Replace with your code snippet

import validmind as vm

vm.init(
    api_host="https://api.prod.validmind.ai/api/v1/tracking",
    api_key="...",
    api_secret="...",
    project="...",
)

<a id='toc4_'></a>

## List and filter available tests

Before we run a test, let's find a suitable metric for this demonstration. Let's assume you want to generate the *pearson correlation matrix* for a dataset. A Pearson correlation matrix is a table that shows the [Pearson correlation coefficients](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) between several variables. 

In the [Explore tests](../explore_tests.ipynb) notebook, we learned how to pass a `filter` to the `list_tests` function. We'll do the same here to find the test ID for the pearson correlation matrix:


In [ ]:
vm.tests.list_tests(filter="PearsonCorrelationMatrix")

From the output, you can see that the test ID for the pearson correlation matrix is `validmind.data_validation.PearsonCorrelationMatrix`. The `describe_test` function gives you more information about the test, including its **Required Inputs**:


In [ ]:
test_id = "validmind.data_validation.PearsonCorrelationMatrix"
vm.tests.describe_test(test_id)

Since this test requires a dataset, it should throw an error if you were to run it without passing a `dataset` input:


In [ ]:
try:
    vm.tests.run_test(test_id)
except Exception as e:
    print(e)

<a id='toc5_'></a>

## Create a sample dataset

Now, let's build a sample dataset so you can generate its pearson correlation matrix. The sklearn `make_classification` function can generate a random dataset for testing:


In [ ]:
import pandas as pd
from sklearn.datasets import make_classification

X, y = make_classification(
    n_samples=10000,
    n_features=10,
    weights=[0.1],
    random_state=42,
)
X.shape
y.shape

df = pd.DataFrame(X, columns=[f"feature_{i}" for i in range(X.shape[1])])
df["target"] = y
df.head()

<a id='toc5_1_'></a>

### Initialize a ValidMind dataset

ValidMind dataset objects provide a wrapper to any type of dataset (NumPy, Pandas, Polars, etc.) so that tests can run transparently regardless of the underlying library. A VM dataset object can be created using the [`init_dataset`](https://docs.validmind.ai/validmind/validmind.html#init_dataset) function from the ValidMind (`vm`) module.

This function takes a number of arguments:

- `dataset`: The raw dataset that you want to provide as input to tests.
- `input_id`: A unique identifier that allows tracking what inputs are used when running each individual test.
- `target_column`: A required argument if tests require access to true values. This is the name of the target column in the dataset.

Below you can see how to initialize a VM dataset for the sample `df` you created previously:


In [ ]:
vm_dataset = vm.init_dataset(
    df,
    input_id="my_demo_dataset",
    target_column="target",
)

<a id='toc5_2_'></a>

### Run tests with the dataset

You can now call `run_test` with the new `vm_dataset` object as input:


In [ ]:
result = vm.tests.run_test(
    test_id,
    inputs={"dataset": vm_dataset},
)

This dataset can also be used for any other test that requires a dataset input. 

- Let's try to find a *class imbalance* to understand the distribution of the target column in the dataset. 
- Class imbalance is a common problem in machine learning, particularly in classification tasks, where the number of instances (or data points) in each class isn't evenly distributed across the available categories.

We'll use `list_tests` again to showcase how to filter tests for tabular data:


In [ ]:
sorted(vm.tests.list_tags())

In [ ]:
vm.tests.list_tests(tags=["binary_classification", "tabular_data"])

<a id='toc5_2_1_'></a>

#### Run a test that accepts parameters

The test ID for the class imbalance test is `validmind.data_validation.ClassImbalance`. If you describe this test you will find that it also accepts some parameters:


In [ ]:
vm.tests.describe_test("validmind.data_validation.ClassImbalance")

The `min_percent_threshold` will allow you configure the threshold for an acceptable class imbalance. 

- Let's run the test without any parameters to see its output using a default value for the threshold. 
- We also call the `log` method on the result to send the results of the tests to the ValidMind platform.


In [ ]:
result = vm.tests.run_test(
    "validmind.data_validation.ClassImbalance",
    inputs={"dataset": vm_dataset},
)

result.log()

This test passes the pass-fail criteria with the default threshold of 10%. 

- Let's try to run the test with a threshold of 20% to see if it fails. Notice the use of the 'custom_threshold' `result_id` in the test ID. 
- This allows you to submit individual results for the same test to the platform, as we'll see in the next section.


In [ ]:
result = vm.tests.run_test(
    "validmind.data_validation.ClassImbalance:custom_threshold",
    inputs={"dataset": vm_dataset},
    params={"min_percent_threshold": 20},
)

result.log()

<a id='toc6_'></a>

## Add test results to documentation

The previous result shows that the test didn't pass the threshold of 20% for class imbalance. With these results logged, you can now add them to your model documentation:

1. In the [Platform UI](https://app.prod.validmind.ai), go to the **Documentation** page for the model you registered earlier.

2. Expand the **2.1. Data Description** section.

3. Hover between any existing content blocks to reveal the **+** button.

4. Click on the **+** button and choose **Test-Driven Block**. This will open a dialog where you can select:
    1. Type: `Threshold Test` 
    2. Threshold Test: `Class Imbalance Custom Threshold Test` 
    - You can preview the result and then click **Insert Block** in the bottom-right corner to add it to the documentation.

<a id='toc7_'></a>

## Next steps

While you can look at the results of this test suite right in the notebook where you ran the code, there is a better way — expand the rest of your documentation in the platform UI and take a look around. 

What you see is the full draft of your model documentation in a more easily consumable version. From here, you can make qualitative edits to model documentation, view guidelines, collaborate with validators, and submit your model documentation for approval when it's ready.

<a id='toc7_1_'></a>

### Discover more learning resources

In an upcoming companion notebook, you'll learn how to run tests that require a dataset and model as inputs. This will allow you to generate documentation for model evaluation metrics such as ROC-AUC, F1 score, etc. for your model.

We also offer many other interactive notebooks to help you document models:

- [Run tests & test suites](https://docs.validmind.ai/guide/testing-overview.html)
- [Code samples](https://docs.validmind.ai/guide/samples-jupyter-notebooks.html)

Or, visit our [documentation](https://docs.validmind.ai/) to learn more about ValidMind.